In [1]:
import numpy as np
import pandas as pd
import tensorflow.compat.v1 as tf
from koopmanlib.dictionary import PsiNNWithEncoder
from koopmanlib.solver import KoopmanDLSolver

2025-08-26 03:07:43.368212: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-08-26 03:07:43.387858: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-26 03:07:43.413973: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-26 03:07:43.420642: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-26 03:07:43.438678: I tensorflow/core/platform/cpu_feature_guar

In [2]:
dim = 3 
hidden_dim = 10  
input_dim = 0
num_visible = 2
num_hidden = 1
hidden_size = 32
kernel_size = 31
n_blocks = 3
n_layers = 1
n_feature = 2
rank = 3
batch_size = 512
n_train = 10000
n_valid = 1000
n_test = 1000
dropout = 0
num_epochs = 20
lamb = 0
learning_rate = 1e-3

In [3]:
X_train = pd.read_csv('KO_train.csv', header=None)
X_valid = pd.read_csv('KO_valid.csv', header=None)
X_test = pd.read_csv('KO_test.csv', header=None)
X_result = np.concatenate([X_train, X_test, X_valid], axis=-1)

In [6]:
idx_list = [[0, 1], [0, 2], [1, 2]]
for idx in idx_list:
    error_temp = []
    X_v_train = X_train.values[idx, :].reshape(2, -1)
    X_v_valid = X_valid.values[idx, :].reshape(2, -1)
    X_v_test = X_test.values[idx, :].reshape(2, -1)
    length = X_train.shape[1] // n_train
    H_train = []
    for i in range(n_train):
        H_train.append(X_v_train[:, i*length:(i+1)*length])
    H_train = np.stack([H_train[idx].T for idx in range(n_train)], axis=0)
    H_valid = []
    for i in range(n_valid):
        H_valid.append(X_v_valid[:, i*length:(i+1)*length])
    H_valid = np.stack([H_valid[idx].T for idx in range(n_valid)], axis=0)
    H_test = []
    for i in range(n_test):
        H_test.append(X_v_test[:, i*length:(i+1)*length])
    H_test = np.stack([H_test[idx].T for idx in range(n_test)], axis=0)
    data_train = [H_train[:, 0:-1,:], H_train[:, 1:,:]]
    data_valid = [H_valid[:, 0:-1,:], H_valid[:, 1:,:]]
    basis_function = PsiNNWithEncoder(
        visible_dim=num_visible,           
        num_hidden=num_hidden,       
        kernel_size=kernel_size,
        layer_sizes=[hidden_dim, hidden_dim, hidden_dim],
        n_psi_train=dim+n_feature,
        target_dim=num_visible
    )
    solver = KoopmanDLSolver(dic=basis_function,
                            target_dim=dim-1,
                            reg=0.0)
    solver.build(data_train=data_train,
                     data_valid=data_valid,
                     epochs=num_epochs,
                     batch_size=batch_size,
                     lr=learning_rate,
                     log_interval=1,
                     lr_decay_factor=0.92)
    for j in range(n_test):
        x_traj = H_test[j, :,:]
        x0_test = x_traj[0]
        x0_test = x0_test.reshape(-1, x0_test.shape[-1])
        x_est_traj_DL = solver.predict(x0_test, length-1)
        error_temp.append((x_est_traj_DL[:, :dim]-x_traj[1:, :dim]) ** 2)
    error = np.mean(np.array(error_temp))
    full_dims = set(range(dim)) 
    missing_dim = list(full_dims - set(idx))[0]
    path = f"error_hidden_{missing_dim}_edmddl.csv"
    df = pd.DataFrame([error])
    df.to_csv(path, index=False)

2025-08-26 03:13:04.861994: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:1756177984.877721    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177984.902606    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177984.903910    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177984.908250    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177984.916549    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177984.932465    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177985.024473    2322 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1756177985.024912    2322 gpu_t

Epoch 1/2


I0000 00:00:1756177987.600574    3979 service.cc:146] XLA service 0x7ef83000e0c0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1756177987.600610    3979 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4090, Compute Capability 8.9
2025-08-26 03:13:07.688103: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


15/20 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - loss: 5.8398e-06

I0000 00:00:1756177990.479999    3979 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


20/20 ━━━━━━━━━━━━━━━━━━━━ 9s 187ms/step - loss: 5.6820e-06 - val_loss: 4.0603e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 3.8589e-06 - val_loss: 3.4227e-06
number of the outer loop: 0
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 31ms/step - loss: 3.0824e-06 - val_loss: 2.3663e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 2.0468e-06 - val_loss: 1.8325e-06
number of the outer loop: 1
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 1.5423e-06 - val_loss: 1.5814e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.4591e-06 - val_loss: 1.5504e-06
number of the outer loop: 2
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.4222e-06 - val_loss: 1.4352e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 1.3740e-06 - val_loss: 1.3916e-06
number of the outer loop: 3
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - loss: 1.2342e-06 - val_loss: 1.3035e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 1.2386e-06 

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'psi_nn_layer_3', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 151ms/step - loss: 3.8016e-05 - val_loss: 1.1736e-05
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 8.4973e-06 - val_loss: 5.0392e-06
number of the outer loop: 0
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 3.5954e-06 - val_loss: 3.0086e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 2.7265e-06 - val_loss: 2.5481e-06
number of the outer loop: 1
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.4582e-06 - val_loss: 2.2354e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 2.1523e-06 - val_loss: 2.1689e-06
number of the outer loop: 2
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 2.0078e-06 - val_loss: 1.9787e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 1.8810e-06 - val_loss: 1.8348e-06
number of the outer loop: 3
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - loss: 1.7376e-06 - val_loss: 1.7519e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1

/usr/local/lib/python3.11/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'psi_nn_layer_4', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 7s 158ms/step - loss: 1.4797e-05 - val_loss: 1.3601e-05
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 1.1814e-05 - val_loss: 1.2207e-05
number of the outer loop: 0
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 9.9368e-06 - val_loss: 7.7879e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 6.9337e-06 - val_loss: 6.5730e-06
number of the outer loop: 1
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 5.3390e-06 - val_loss: 4.7433e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 4.5573e-06 - val_loss: 4.3434e-06
number of the outer loop: 2
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 3.6314e-06 - val_loss: 3.2167e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - loss: 3.1939e-06 - val_loss: 3.3288e-06
number of the outer loop: 3
Epoch 1/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 2.6946e-06 - val_loss: 2.6249e-06
Epoch 2/2
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 2